# Memory profiling of MPI-parallel runs

## Initialization

In order to execute the individual solver runs,
we are going to employ the mini batch processor,
for running the calculations on the local machine.
We also have to initialize the workflow management system and create 
a database.

Note: 
1. This tutorial can be found in the source code repository as as `convStudy.ipynb`. 
   One can directly load this into Jupyter to interactively work with the following code examples.
2. **In the following line, the reference to `BoSSSpad.dll` is required**. 
   You must either set `#r "BoSSSpad.dll"` to something which is appropirate for your computer
   (e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if you installed the binary distribution),
   or, if you are working with the source code, you must compile `BoSSSpad` and put it side-by-side to this worksheet file
   (from the original location in the repository, you can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).

In [1]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

The below script needs to be able to find the current output cell; this is an easy method to get it.


(1,1): error CS0006: Metadata file 'BoSSSpad.dll' could not be found

(5,7): error CS0246: The type or namespace name 'ilPSP' could not be found (are you missing a using directive or an assembly reference?)

(6,7): error CS0246: The type or namespace name 'ilPSP' could not be found (are you missing a using directive or an assembly reference?)

(7,7): error CS0246: The type or namespace name 'BoSSS' could not be found (are you missing a using directive or an assembly reference?)

(8,7): error CS0246: The type or namespace name 'BoSSS' could not be found (are you missing a using directive or an assembly reference?)

(9,7): error CS0246: The type or namespace name 'BoSSS' could not be found (are you missing a using directive or an assembly reference?)

(10,7): error CS0246: The type or namespace name 'BoSSS' could not be found (are you missing a using directive or an assembly reference?)

(11,7): error CS0246: The type or namespace name 'BoSSS' could not be found (are you missing a using

Error: compilation error